<a href="https://colab.research.google.com/github/2303a52157/Generative-AI_2025/blob/main/Gen_AI_Lab06_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import RMSprop
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error
import tensorflow as tf
from keras.models import load_model

def root_mean_absolute_error(y_true, y_pred):

    y_true = tf.cast(y_true, y_pred.dtype)
    return tf.sqrt(tf.reduce_mean(tf.abs(y_true - y_pred)))

file_path = '/content/Housing (2).csv'
data = pd.read_csv(file_path)


X = data.drop('price', axis=1)
y = data['price']

categorical_features = X.select_dtypes(include=['object']).columns
numerical_features = X.select_dtypes(include=[np.number]).columns

preprocessor = ColumnTransformer(transformers=[
    ('num', StandardScaler(), numerical_features),
    ('cat', OneHotEncoder(), categorical_features)])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

pipeline = Pipeline(steps=[('preprocessor', preprocessor)])
X_train = pipeline.fit_transform(X_train)
X_test = pipeline.transform(X_test)

model = Sequential()
model.add(Dense(18, activation='swish', input_dim=X_train.shape[1]))
model.add(Dense(26, activation='swish'))
model.add(Dense(20, activation='swish'))
model.add(Dense(15, activation='swish'))
model.add(Dense(1, activation='linear'))

model.compile(loss='mean_squared_error', optimizer=RMSprop(), metrics=[root_mean_absolute_error])

history = model.fit(X_train, y_train, epochs=200, batch_size=64, validation_data=(X_test, y_test))

model.save('housing_price_model.h5')

y_train_pred = model.predict(X_train)
y_test_pred = model.predict(X_test)

train_rmse = np.sqrt(mean_squared_error(y_train, y_train_pred))
test_rmse = np.sqrt(mean_squared_error(y_test, y_test_pred))

print(f"Training RMSE: {train_rmse}")
print(f"Testing RMSE: {test_rmse}")

loaded_model = load_model('housing_price_model.h5', custom_objects={"root_mean_absolute_error": root_mean_absolute_error})
def predict_housing_price(features):

    features_df = pd.DataFrame([features], columns=['area', 'bedrooms', 'bathrooms', 'stories', 'parking', 'mainroad', 'guestroom', 'basement', 'hotwaterheating', 'airconditioning', 'furnishingstatus'])

    features_df['prefarea'] = 'no'

    features = pipeline.transform(features_df)
    prediction = loaded_model.predict(features)
    return prediction[0][0]

user_input = [8000, 3, 2, 2, 1, 'yes', 'no', 'no', 'yes', 'no', 'furnished']
predicted_price = predict_housing_price(user_input)
print(f"Predicted Housing Price: {predicted_price}")

Epoch 1/200


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


7/7 ━━━━━━━━━━━━━━━━━━━━ 2s 48ms/step - loss: 24637291364352.0000 - root_mean_absolute_error: 2154.5693 - val_loss: 30129990402048.0000 - val_root_mean_absolute_error: 2258.3000
Epoch 2/200
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - loss: 24292525867008.0000 - root_mean_absolute_error: 2152.0005 - val_loss: 30129986207744.0000 - val_root_mean_absolute_error: 2258.2998
Epoch 3/200
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: 23979469307904.0000 - root_mean_absolute_error: 2146.2109 - val_loss: 30129982013440.0000 - val_root_mean_absolute_error: 2258.2998
Epoch 4/200
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 25394025594880.0000 - root_mean_absolute_error: 2175.1089 - val_loss: 30129971527680.0000 - val_root_mean_absolute_error: 2258.2998
Epoch 5/200
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 24387344400384.0000 - root_mean_absolute_error: 2153.2061 - val_loss: 30129963139072.0000 - val_root_mean_absolute_error: 2258.2993
Epoch 6/200
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - loss: 2570

14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step 
Training RMSE: 4442759.91955271
Testing RMSE: 4895414.685431256


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 137ms/step
Predicted Housing Price: 741805.0625
